*PROJECT : Federated Learning Trusted Aggregator*

In [1]:
!pip install syft

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0721 11:45:25.565339 140122342528896 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0721 11:45:25.586005 140122342528896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
#create a couple of workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


In [4]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0721 11:45:26.638113 140122342528896 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0721 11:45:26.640689 140122342528896 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0721 11:45:26.641965 140122342528896 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0721 11:45:26.642900 140122342528896 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0721 11:45:26.644229 140122342528896 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0721 11:45:26.645990 140122342528896 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset

data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by
# by sending some trainig data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alice_data = data[0:2].send(alice)
alice_target = target[0:2].send(alice)

In [0]:
#initialize a toy model
model = nn.Linear(2,1)

In [20]:
for round_iter in range(10):
  bobs_model = model.copy().send(bob)
  alice_model = model.copy().send(alice)

  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alice_opt = optim.SGD(params=alice_model.parameters(), lr=0.1)
  
  for i in range(10):
    
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) **2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
  
    alice_opt.zero_grad()
    alice_pred = alice_model(alice_data)
    alice_loss = ((alice_pred - alice_target) **2).sum()
    alice_loss.backward()

    alice_opt.step()
    alice_loss = alice_loss.get().data
    alice_loss
    
  alice_model.move(secure_worker)
  bobs_model.move(secure_worker)
  
#   model.weight.set_(((alice_model.weight.data + bobs_model.weight.data) / 2).get())
#   model.bias.data.set_(((alice_model.bias.data + bobs_model.bias.data) / 2).get())
  
  print("Bob:" + str(bobs_loss) + "Alice:" + str(alice_loss))

Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)
Bob:tensor(0.0051)Alice:tensor(0.0051)


In [0]:
alice_model.move(secure_worker)
bobs_model.move(secure_worker)

In [13]:
secure_worker._objects

{39226260701: Parameter containing:
 tensor([[-0.0359,  0.0911]], requires_grad=True),
 49610365891: Parameter containing:
 tensor([[-0.0359,  0.0911]], requires_grad=True),
 54326023721: Parameter containing:
 tensor([-0.0557], requires_grad=True),
 63493462574: Parameter containing:
 tensor([-0.0557], requires_grad=True)}

In [14]:
(alice_model.weight.data + bobs_model.weight.data) / 2 #get the average

(Wrapper)>[PointerTensor | me:13755231545 -> secure_worker:69198149712]

In [0]:
#model.weight.set_(((alice_model.weight.data + bobs_model.weight.data) / 2).get())

In [0]:
#model.bias.data.set_(((alice_model.bias.data + bobs_model.bias.data) / 2).get())